# Google Stock Predicition using LSTM

## Necessary downloads

In [ ]:
!pip install pydot
!pip install graphviz

## Necessary Imports

In [ ]:
# Basic data manipulation packages
import numpy as np 
import pandas as pd 
import os

# Visualization
import matplotlib.pyplot as plt
from keras.utils import plot_model

# Data Pre-processing
from sklearn.preprocessing import MinMaxScaler

# LSTM Model Construction
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


## Loading and Pre-processing the data

### Load the training data

In [ ]:
dataset_train = pd.read_csv("../input/googledta/trainset.csv")
dataset_train

In [ ]:
trainset = dataset_train.iloc[:,1:2].values
trainset

### Perform Min Max scaling 

In [ ]:
sc = MinMaxScaler(feature_range = (0,1))
training_scaled = sc.fit_transform(trainset)
training_scaled

### Split data in 60 parts for LSTM input

In [ ]:
x_train = []
y_train = []
for i in range(60,1259):
    x_train.append(training_scaled[i-60:i, 0])
    y_train.append(training_scaled[i,0])
x_train,y_train = np.array(x_train),np.array(y_train)

In [ ]:
x_train.shape

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
print(x_train.shape)

In [ ]:
## LSTM Model

### Model Constrcuction

In [ ]:
model = Sequential()
model.add(LSTM(units = 50,return_sequences = True,input_shape = (x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units = 50,return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50,return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
model.add(Dense(units = 1))
model.summary()

### Model Structure

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

### Training the LSTM Model

In [ ]:
model.compile(optimizer = 'adam',loss = 'mean_squared_error')
model.fit(x_train,y_train,epochs = 100, batch_size = 32)

## Testing the trained model

### Loading the test data

In [ ]:
dataset_test = pd.read_csv("../input/googledta/testset.csv")
real_stock_price = dataset_test.iloc[:,1:2].values
dataset_total = pd.concat((dataset_train['Open'],dataset_test['Open']),axis = 0)
dataset_total

### Pre-processing the data

In [ ]:
inputs = dataset_total[len(dataset_total) - len(dataset_test)-60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
inputs.shape

In [ ]:
x_test = []
for i in range(60,185):
    x_test.append(inputs[i-60:i,0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
x_test.shape

### Predict the prices 

In [ ]:
predicted_price = model.predict(x_test)
predicted_price = sc.inverse_transform(predicted_price)
predicted_price

### model predicition visualization

In [ ]:
plt.plot(real_stock_price,color = 'green', label = 'Actual Price')
plt.plot(predicted_price, color = 'red', label = 'Predicted Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()